In [24]:
import numpy as np
import open3d as o3d
from scipy.optimize import least_squares
from sklearn.cluster import DBSCAN

In [32]:
pcd = o3d.io.read_point_cloud("filtered.pcd")
downpcd = pcd.voxel_down_sample(voxel_size=0.000001)
points = np.asarray(downpcd.points)
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
normals = np.asarray(downpcd.normals)

In [33]:
# Load your point cloud
point_cloud = o3d.io.read_point_cloud("filtered.pcd")
point_cloud.points = o3d.utility.Vector3dVector(points)  # Replace 'points' with your point cloud data

# Estimate normals for better visualization and context (optional but useful)
point_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=20))

# Use Alpha Shape to create a concave hull of the point cloud (the outer boundary)
alpha = 0.0045  # Alpha value controls the tightness of the hull; adjust based on your data
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(point_cloud, alpha)

# Extract vertices (points) of the hull as the outer surface points
surface_points = np.asarray(mesh.vertices)

# Visualize the extracted surface points
surface_cloud = o3d.geometry.PointCloud()
surface_cloud.points = o3d.utility.Vector3dVector(surface_points)

# Optional: Estimate normals for the surface points for further processing or visualization
surface_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Visualize the surface points to ensure they correctly represent the outer shell
o3d.visualization.draw_geometries([surface_cloud], point_show_normal=True)

In [34]:
import numpy as np
from sklearn.decomposition import PCA

# Estimate normals with tuned parameters
surface_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Convert Open3D points to NumPy array for PCA
points_np = np.asarray(downpcd.points)

# Perform PCA to find the centroid and main components
pca = PCA(n_components=1)
pca.fit(points_np)
centroid = np.mean(points_np, axis=0)

# Convert normals to NumPy array
normals_np = np.asarray(surface_cloud.normals)

# Reorient normals to point outward from the centroid
for i in range(len(normals_np)):
    normal = normals_np[i]
    vector_to_point = points_np[i] - centroid
    # Flip the normal if it is pointing inward
    if np.dot(normal, vector_to_point) < 0:
        normals_np[i] = -normal

# Update the point cloud normals with the reoriented normals
surface_cloud.normals = o3d.utility.Vector3dVector(normals_np)
normals = np.asarray(surface_cloud.normals)
points = np.asarray(surface_cloud.points)

# Visualize the reoriented normals
o3d.visualization.draw_geometries([surface_cloud], point_show_normal=True)

## Regular RANSAC Cylinder

In [49]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.cluster import DBSCAN

# Function to apply DBSCAN for connectivity check
def apply_connectivity_check(inliers, eps=0.5, min_samples=5):
    """
    Apply DBSCAN to ensure the inliers form a connected region.
    
    Parameters:
        inliers (ndarray): Nx3 array of inlier point coordinates.
        eps (float): Maximum distance between points for them to be considered in the same neighborhood.
        min_samples (int): Minimum number of points to form a cluster.
    
    Returns:
        largest_cluster (ndarray): The largest contiguous cluster of points.
    """
    if len(inliers) == 0:
        return np.array([])  # Return empty if no inliers

    db = DBSCAN(eps=eps, min_samples=min_samples).fit(inliers)
    labels = db.labels_

    # Find the largest cluster (most common label excluding noise)
    unique_labels, counts = np.unique(labels[labels >= 0], return_counts=True)
    if len(unique_labels) == 0:
        return np.array([])  # No valid clusters found
    
    largest_cluster_label = unique_labels[np.argmax(counts)]
    largest_cluster = inliers[labels == largest_cluster_label]

    return largest_cluster

# RANSAC function for fitting a cylinder, now with DBSCAN connectivity check
def ransac_cylinder(points, threshold, iterations, eps=0.01, min_samples=5, min_radius=None, max_radius=None):
    """
    RANSAC-based cylinder fitting with DBSCAN-based connectivity check.

    Parameters:
        points (ndarray): Nx3 array of point coordinates.
        threshold (float): Distance threshold to consider a point as an inlier.
        iterations (int): Number of RANSAC iterations.
        eps (float): Maximum distance between points for connectivity in DBSCAN.
        min_samples (int): Minimum number of points for DBSCAN clusters.
        min_radius (float): Minimum allowed radius for the cylinder (optional).
        max_radius (float): Maximum allowed radius for the cylinder (optional).

    Returns:
        best_params (list): Parameters of the best-fit cylinder (center, axis, radius).
        best_inliers (ndarray): Largest connected inlier points that fit the cylinder model.
    """
    best_inliers = []
    best_params = None
    best_score = 0

    for i in range(iterations):
        # Randomly sample 5 points to define a cylinder
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_cylinder(sample)
        if params is None:
            continue

        x0, y0, z0, a, b, c, r = params
        if (min_radius is not None and r < min_radius) or (max_radius is not None and r > max_radius):
            continue

        cylinder_center = np.array([x0, y0, z0])
        cylinder_axis = np.array([a, b, c])
        distances = distance_to_cylinder(points, cylinder_center, cylinder_axis, r)
        inliers = points[distances < threshold]

        if len(inliers) == 0:
            continue  # Skip if no inliers found

        # Apply DBSCAN to check for connectivity and get the largest cluster
        connected_inliers = apply_connectivity_check(inliers, eps=eps, min_samples=min_samples)

        if len(connected_inliers) > best_score:
            best_score = len(connected_inliers)
            best_inliers = connected_inliers
            best_params = params

    return best_params, best_inliers

# Supporting functions: Fit cylinder, distance to cylinder
def fit_cylinder(points, min_radius=None, max_radius=None):
    x0, y0, z0 = np.mean(points, axis=0)
    initial_axis = np.array([0, 0, 1])
    initial_radius = np.mean(np.linalg.norm(points - np.array([x0, y0, z0]), axis=1))
    initial_guess = [x0, y0, z0, *initial_axis, initial_radius]
    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x if result.success else None

def residuals(params, points):
    x0, y0, z0, a, b, c, r = params
    axis_direction = np.array([a, b, c])
    axis_direction /= np.linalg.norm(axis_direction)
    vector_to_points = points - np.array([x0, y0, z0])
    projection_length = np.dot(vector_to_points, axis_direction)
    projection = projection_length[:, np.newaxis] * axis_direction
    perpendicular_distance = np.linalg.norm(vector_to_points - projection, axis=1)
    return perpendicular_distance - r

def distance_to_cylinder(points, cylinder_center, axis_direction, radius):
    """
    Calculate the Euclidean distance of points to a cylinder defined by its center, axis direction, and radius.
    """
    vector_to_points = points - cylinder_center
    projection_length = np.dot(vector_to_points, axis_direction)
    projection = projection_length[:, np.newaxis] * axis_direction
    perpendicular_distance = np.linalg.norm(vector_to_points - projection, axis=1)
    return np.abs(perpendicular_distance - radius)

## Plane

In [ ]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.cluster import DBSCAN

# Function to apply DBSCAN for connectivity check
def apply_connectivity_check(inliers, eps=0.5, min_samples=5):
    """
    Apply DBSCAN to ensure the inliers form a connected region.
    
    Parameters:
        inliers (ndarray): Nx3 array of inlier point coordinates.
        eps (float): Maximum distance between points for them to be considered in the same neighborhood.
        min_samples (int): Minimum number of points to form a cluster.
    
    Returns:
        largest_cluster (ndarray): The largest contiguous cluster of points.
    """
    if len(inliers) == 0:
        return np.array([])  # Return empty if no inliers

    db = DBSCAN(eps=eps, min_samples=min_samples).fit(inliers)
    labels = db.labels_

    # Find the largest cluster (most common label excluding noise)
    unique_labels, counts = np.unique(labels[labels >= 0], return_counts=True)
    if len(unique_labels) == 0:
        return np.array([])  # No valid clusters found
    
    largest_cluster_label = unique_labels[np.argmax(counts)]
    largest_cluster = inliers[labels == largest_cluster_label]

    return largest_cluster

# Function to fit a plane using least squares
def fit_plane(points):
    """
    Fit a plane to the given set of points using least squares.
    
    Parameters:
        points (ndarray): Nx3 array of points.
    
    Returns:
        plane_params (tuple): A tuple (a, b, c, d) for the plane equation ax + by + cz + d = 0.
    """
    centroid = np.mean(points, axis=0)
    shifted_points = points - centroid
    _, _, vh = np.linalg.svd(shifted_points)
    normal = vh[-1, :]
    d = -np.dot(normal, centroid)
    return (*normal, d)

# Function to calculate the Euclidean distance from a point to a plane
def distance_to_plane(points, plane_params):
    """
    Calculate the Euclidean distance of points to a plane.
    
    Parameters:
        points (ndarray): Nx3 array of points.
        plane_params (tuple): A tuple (a, b, c, d) for the plane equation ax + by + cz + d = 0.
    
    Returns:
        distances (ndarray): Distances of each point to the plane.
    """
    a, b, c, d = plane_params
    plane_normal = np.array([a, b, c])
    distances = np.abs(np.dot(points, plane_normal) + d) / np.linalg.norm(plane_normal)
    return distances

# RANSAC function for fitting a plane, now with DBSCAN connectivity check
def ransac_plane(points, threshold, iterations, eps=0.5, min_samples=5):
    """
    RANSAC-based plane fitting with DBSCAN-based connectivity check.

    Parameters:
        points (ndarray): Nx3 array of point coordinates.
        threshold (float): Distance threshold to consider a point as an inlier.
        iterations (int): Number of RANSAC iterations.
        eps (float): Maximum distance between points for connectivity in DBSCAN.
        min_samples (int): Minimum number of points for DBSCAN clusters.

    Returns:
        best_params (tuple): Parameters of the best-fit plane (a, b, c, d).
        best_inliers (ndarray): Largest connected inlier points that fit the plane model.
    """
    best_inliers = []
    best_params = None
    best_score = 0

    for i in range(iterations):
        # Randomly sample 3 points to define a plane
        sample = points[np.random.choice(points.shape[0], 3, replace=False)]
        params = fit_plane(sample)

        distances = distance_to_plane(points, params)
        inliers = points[distances < threshold]

        if len(inliers) == 0:
            continue  # Skip if no inliers found

        # Apply DBSCAN to check for connectivity and get the largest cluster
        connected_inliers = apply_connectivity_check(inliers, eps=eps, min_samples=min_samples)

        if len(connected_inliers) > best_score:
            best_score = len(connected_inliers)
            best_inliers = connected_inliers
            best_params = params

    return best_params, best_inliers

In [52]:
best_params, best_inliers = ransac_cylinder(points, threshold= 0.01, iterations= 1000, min_radius=0 , max_radius=0.1)

In [53]:
cylinder_cloud = o3d.geometry.PointCloud()
cylinder_cloud.points = o3d.utility.Vector3dVector(best_inliers)
cylinder_cloud.paint_uniform_color([0,1,0])

surface_cloud.paint_uniform_color([0,0,0])

o3d.visualization.draw_geometries([surface_cloud, cylinder_cloud], point_show_normal=True)

In [9]:
# Function to remove inliers and store them for later use
def remove_inliers(points, inliers):
    """
    Removes inliers from the original point cloud and returns the remaining points.
    
    Parameters:
        points (ndarray): Nx3 array of point coordinates.
        inliers (ndarray): Nx3 array of inlier points.
    
    Returns:
        remaining_points (ndarray): Points that were not inliers.
    """
    # Find the remaining points (outliers)
    remaining_points = np.array([point for point in points if point.tolist() not in inliers.tolist()])
    return remaining_points


In [20]:
remaining_points = remove_inliers(points, best_inliers)

In [21]:
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(remaining_points)

In [22]:
o3d.visualization.draw_geometries([point_cloud], point_show_normal=True)